# TP5 - Modelos Baseados em Dados

Este trabalho prático visa demonstrar alguns exemplos de modelação baseada em dados, usando métodos de classificação (Parte 1) e regressão (Parte 2).

Os modelos QSAR são de grande utilizade na previsão da actividade química e biológica de compostos. A sua utilização na previsão de efeitos toxicológicos tem sido impulsionada nos últimos anos enquanto alternativa ao sacríficio de animais em ensaios *in vivo*, em parte através da acção do *European Union Reference Laboratory for Alternatives to Animal Testing* localizado em Ispra, Itália. Do mesmo modo, a possibilidade de avaliar *in silico* a actividade de milhares de compostos (incluíndo compostos ainda não sintetizados) possibilita uma poupança anual da ordem dos milhões de euros, tanto em equipamento e recursos humanos, como em reagentes (com adicionais benefícios ao nível da remediação da produção de resítudos).

Regra geral, um estudo QSAR é composto pelas seguintes etapas:
1. Colheita dos dados (actividade por composto)
1. Curação da base de dados (eliminação de entradas duplicadas, confirmação de resultados consultando a bibliografia original, etc)
1. Geração de descritores a partir da estrutura dos compostos.
1. Divisão dos dados entre treino e teste.
1. Treino do algoritmo escolhido (eventualmente incluindo a optimização dos hiper-parâmetros desse algoritmo) usando os dados de treino
1. Avaliação da capacidade preditiva do algoritmo usando os dados de teste.

Adicionalmente, também é possível fazer a "autópsia" do modelo, de forma a conseguir racionalizar os processos inerentes à actividade estudada. Do mesmo modo, após a geração dos edscritores moleculares, é boa prática proceder a uma caracterização estatística dos mesmos, de forma a diagnosticar de antemão casos extremos que poderão ser *outliers* do modelo, ou de alguma forma enviesar o algoritmo.

Em cada uma das partes deste trabalho, começamos com uma base de dados já curada e usada em pelo menos uma publicação científica revista por pares. O protocolo de cada etapa destes trabalhos práticos inicia-se então na geração de descritores. Em cada parte do trabalho, o problema será primeiro abordado usando uma técnica linear (QSAR clássico) e depois usando uma técnica de Apendizagem Automática (*Machine Learning*).

Este _notebook_ encontra-se optimizado para correr usando a plataforma Colab da Google, mas pode ser usado em qualquer computador com o Jupyter Lab ou Jupyter Notebook e uma instalação dos seguintes pacotes.
* `numpy`
* `pandas`
* `scikit-learn`
* `seaborn`
* `rdkit`

Na plataforma Colab, apenas o rdkit requer instalação, pelo que as instruções para instalação dos demais pacotes encontra-se comentada. Caso deseje instalar todos os pacotes, deverá remover o sinal cardinal (#) do início das primeiras 4 linhas da célula de código abaixo.

In [ ]:
#! pip install numpy
#! pip install pandas
#! pip install scikit-learn
#! pip install seaborn
! pip install rdkit

## Parte 1 - Modelo classificador da capacidade mutagénica de aminas simples.

Esta parte do trabalho usa dados originalmente publicados em:

Benigni, R.; Bossa, C.; Netzeva, T. I.; Rodomonte, A.; Tsakovska, I. "Mechanistic QSAR of aromatic amines: New models for discriminating between homocyclic mutagens and nonmutagens, and validation of models for carcinogens." _Environ. Mol. Mutagen._ 2007, **48**, 754–771. http://dx.doi.org/10.1002/em.20355

Este trabalho pretende treinar um modelo classificador da actividade mutagénica de aminas simples, com base em dados recolhidos para as estirpes TA98 e TA100 da _Salmonella thyphimurium_. Os dados experimentais disponíveis são gerados num fiheiro do tipo `csv` na célula seguinte (que também importa os pacotes necessários para esta parte do trabalho).

In [ ]:
import time
import numpy as np
import pandas as pd
import seaborn as sns

import matplotlib.pyplot as plt

from rdkit import Chem
from rdkit.Chem import Descriptors
from rdkit.Chem import Draw

with open('parte1-dados.csv','w') as f:
  f.write("""CId,Mutagenic,SMILES
1	 ,1,Nc1ccc2cccc3c2c1-c1ccccc1-3
2	 ,1,Cc1ccc2ccccc2c1N
3	 ,1,Nc1cccc2cc3ccccc3cc12
4	 ,1,Nc1cccc2ccccc12
5	 ,1,Nc1cccc2c1[nH]c1ccccc12
6	 ,1,Nc1cccc2c1Cc1ccccc1-2
7	 ,1,Nc1cccc2c1ccc1ccccc12
8	 ,1,Nc1ccc2ccc3cccc4ccc1c2c34
9	 ,0,Nc1ccccc1-c1ccccc1N
10 ,1,Cc1cc(C)c(N)cc1C
11 ,0,CC(C)c1ccc(N)cc1N
12 ,1,Nc1ccc(F)cc1F
13 ,1,Cc1ccc(N)c(C)c1
14 ,0,Nc1ccc(-c2ccccc2N)cc1
15 ,1,Nc1cc(Cl)c(N)c(Cl)c1
16 ,1,Cc1c(N)ccc2ccccc12
17 ,1,Cc1cc2ccccc2cc1N.Cl
18 ,1,Nc1cc(Cl)ccc1O
19 ,1,Cc1ccc(O)c(N)c1
20 ,1,Nc1ccc2cc3ccccc3cc2c1
21 ,1,Nc1ccccc1-c1ccccc1
22 ,1,Nc1ccc2c(c1)[nH]c1ccccc12
23 ,1,Nc1cc2c3c(cccc3c1)-c1ccccc1-2
24 ,1,Nc1ccc2c(c1)Cc1ccccc1-2
25 ,1,Nc1ccc2ccccc2c1
26 ,1,Nc1ccc2c(ccc3ccccc32)c1
27 ,1,Nc1cc2ccc3cccc4ccc(c1)c2c34
28 ,1,CCc1cc(Cl)ccc1N
29 ,1,Cc1cc(Br)ccc1N
30 ,1,Cc1cc(Cl)ccc1N
31 ,0,Nc1cccc(-c2cccc(N)c2)c1
32 ,1,Nc1ccc(-c2ccc(N)c(Cl)c2)cc1Cl
33 ,1,COc1cc(-c2ccc(N)c(OC)c2)ccc1N
34 ,1,Nc1ccc(-c2cccc(N)c2)cc1
35 ,1,Cc1ccc(-c2ccccc2)cc1N
36 ,0,Nc1cccc(C(F)(F)F)c1
37 ,1,Nc1ccc2[nH]c3ccccc3c2c1
38 ,1,Nc1ccc2c3c(cccc13)-c1ccccc1-2
39 ,1,Nc1ccc2c(c1)-c1ccccc1C2
40 ,1,Nc1ccc2ccc3ccccc3c2c1
41 ,1,Nc1cnc2ccccc2c1
42 ,1,Nc1ccc(CCc2ccc(N)cc2)cc1
43 ,1,CCc1cc(Cc2ccc(N)c(CC)c2)ccc1N
44 ,1,Nc1ccc(Cc2ccc(N)c(F)c2)cc1F
45 ,0,CC(C)c1cc(Cc2ccc(N)c(C(C)C)c2)ccc1N
46 ,1,Nc1ccc(Cc2ccc(N)cc2)cc1
47 ,1,Nc1ccc(SSc2ccc(N)cc2)cc1
48 ,1,Cc1cc(-c2ccccc2)ccc1N
49 ,1,Nc1ccc(-c2ccccc2)cc1
50 ,1,Nc1cccc2[nH]c3ccccc3c12
51 ,1,Nc1cccc2c1-c1ccccc1C2
52 ,1,Nc1cccc2ccc3ccccc3c12
53 ,1,Nc1ccc(Oc2ccc(N)cc2)cc1
54 ,1,Nc1ccc(Sc2ccc(N)cc2)cc1
55 ,1,Nc1cc2cccc3ccc4cccc1c4c32
56 ,0,Nc1ccc(Br)cc1
57 ,1,Nc1ccc(Cl)cc1N
58 ,1,Nc1ccc(C2CCCCC2)cc1
59 ,1,CCOc1ccc(N)cc1
60 ,1,COc1ccc(N)c(C)c1
61 ,0,Cc1ccc(N)c(Cl)c1
62 ,1,Cc1ccc(N)c(Cl)c1
63 ,1,Nc1ccc(Oc2ccccc2)cc1
64 ,1,Nc1cccc2ncccc12
65 ,1,Nc1cc2c3ccccc3ccc2c2ccccc12
66 ,1,Nc1ccc2ncccc2c1
67 ,1,Nc1cccc2c1-c1cccc3cccc-2c13
68 ,1,Nc1ccc2c(c1)-c1cccc3cccc-2c13
69 ,1,Nc1cccc2cccnc12
70 ,1,Nc1c2ccccc2cc2ccccc12
71 ,1,Nc1cc2ccccc2c2ccccc12
72 ,1,Nc1ccc(-c2ccc(N)cc2)cc1
73 ,0,COc1ccc(N)cc1
74 ,0,COc1cccc(N)c1
75 ,0,Nc1ccccc1
76 ,0,Nc1cccc(Cl)c1
77 ,0,CCOc1cccc(N)c1
78 ,0,CCOc1ccccc1N
79 ,0,Nc1ccc(O)cc1
80 ,0,Nc1cccc(O)c1
81 ,0,CC(C)c1cc(Cc2cc(C(C)C)c(N)c(C(C)C)c2)cc(C(C)C)c1N
82 ,0,CCc1cc(Cc2cc(CC)c(N)c(CC)c2)cc(CC)c1N
83 ,0,Cc1cc(Cc2cc(C)c(N)c(C(C)(C)C)c2)cc(C(C)(C)C)c1N
84 ,0,Cc1cc(Cc2cc(C)c(N)c(C(C)C)c2)cc(C(C)C)c1N
85 ,0,CCc1cc(Cc2cc(C)c(N)c(CC)c2)cc(C)c1N
86 ,0,Cc1cc(Cc2cc(C)c(N)c(C)c2)cc(C)c1N
87 ,0,Nc1cccc(-c2ccccc2)c1
88 ,0,Nc1cccc(-c2ccccc2N)c1
89 ,0,COc1ccccc1N
90 ,0,Nc1ccccc1O
91 ,0,Cc1cc(C)c(N)c(C)c1.Cl
92 ,0,Nc1c(Br)cc(Br)cc1Br
93 ,0,Nc1c(Cl)cc(Cl)cc1Cl
94 ,0,CCc1cccc(CC)c1N
95 ,0,Cc1cc(C)cc(N)c1
96 ,0,Cc1cccc(C)c1N
97 ,0,Nc1ccc(Br)cc1Br
98 ,0,Nc1ccc(Cl)cc1Cl
99 ,0,Nc1ccc(I)cc1
100,0,Nc1ccccc1I
101,0,Nc1ccccc1F
102,0,Nc1ccccc1Br
103,0,CCc1ccc(N)cc1
104,0,CCc1ccccc1N
105,0,Cc1ccc(N)cc1
106,0,Cc1ccccc1N
107,0,Cc1ccccc1N
108,0,CCCCc1ccc(-c2ccc(N)cc2)cc1
109,0,CC(C)(C)c1ccc(-c2ccc(N)cc2)cc1
110,0,Nc1ccc(-c2ccc(C(F)(F)F)cc2)cc1
111,0,Nc1ccc(-c2cccc(C(F)(F)F)c2)cc1
""")


### Geração dos Descritores

A Geração dos descritores moleculares é feita usando o pacote `rdkit`. A lista completa dos descritores disponíveis pode ser consultada em https://www.rdkit.org/docs/GettingStartedInPython.html#list-of-available-descriptors. Para este trabalho, iremos usar apenas um conjunto limitado de descritores:
* _MolLogP_
* _BalabanJ_
* _NumHAcceptors_
* _NumHDonors_
* _TPSA_
* _FractionCSP3_

Para isto, primeiro abrimos o ficheiro `parte1-dados.csv` com o pandas, de forma a criar a a _DataFrame_ `dados`. Depois criamos uma coluna vazia para cada um dos descritores moleculares. Finalmente, varremos os dados linha-por-linha, criando uma representação molecular do rdkit a partir do SMILES, e calculando os descritores para cada molécula.

In [ ]:
dados = pd.read_csv('parte1-dados.csv')

# criar colunas vazias para os descritores
dados['MolLogP'] = np.zeros(len(dados))
dados['BalabanJ'] = np.zeros(len(dados))
dados['NumHAcceptors'] = np.zeros(len(dados))
dados['NumHDonors'] = np.zeros(len(dados))
dados['TPSA'] = np.zeros(len(dados))
dados['FractionCSP3'] = np.zeros(len(dados))

for i,line in dados.iterrows():
  m = Chem.MolFromSmiles(line['SMILES'])
  dados.loc[i,'MolLogP'] = Descriptors.MolLogP(m)
  dados.loc[i,'BalabanJ'] = Descriptors.BalabanJ(m)
  dados.loc[i,'NumHAcceptors'] = Descriptors.NumHAcceptors(m)
  dados.loc[i,'NumHDonors'] = Descriptors.NumHDonors(m)
  dados.loc[i,'TPSA'] = Descriptors.TPSA(m)
  dados.loc[i,'FractionCSP3'] = Descriptors.FractionCSP3(m)


dados

As boas práticas recomendam fazer uma análise descritiva prévia dos dados. Neste caso, essa análise limitar-se-á a:
* Tabela de Frequências da variável `Mutagenic` 
* BoxPlot dos descritores
* Inspecção visual da matriz de correlação dos descritores

In [ ]:
# Tabela de frequências para a variável Mutagenic
print(dados.value_counts(subset='Mutagenic'))

Os dados parecem estar aceitavelmente equilibrados entre casos positivos (mutagénicos) e negativos (não-mutagénicos).

In [ ]:
# BoxPlot dos descritores moleculares, usando o seaborn
descritores=['MolLogP','BalabanJ','NumHAcceptors','NumHDonors','TPSA','FractionCSP3']
sns.boxplot(x='variable', y='value',data=dados[descritores].melt())

A análise do boxplot indica que uma veriável (TPSA) tem uma amplitude muito diferente das demais. Isto sugere que o modelo pode benificiar de uma normalização prévia dos dados de entrada (de forma a que todas as variáveis fiquem limitadas ao intervalo [-1:+1]).

In [ ]:
# Heatmap da matriz de correlação
sns.heatmap(dados[descritores+['Mutagenic']].corr(), cmap="RdYlBu", annot=True)

A inspecção da matriz de correlação indica que há três variáveis significativamente correlacionadas (TPSA, NumHDonors e NumHAcceptors). A mesma análise também sugere que a variável-alvo (Mutagenic) parece estar negativamente correlacionada (embora de forma fraca) com a FractionCSP3 e (em menor medida) BalabanJ.

### Treino e Avaliação de um Modelo Linear (Regressão Logística)

Agora que temos os nossos dados gerados e pré-analisados, iremos criar um primeiro modelo de classificação. Embora tenha a palavra "Regressão" no nome, a regressão logística é um método de classificação que tenta encontrar fronteiras lineares nos dados de entrada (_features_) que separem as entradas das várias classes da variável-alvo (_target_).

Tal como foi sugerido na análise preliminar, estes dados devem ser primeiro normalizados. Para isto definimos o nosso modelo como uma "canalização" (_pipeline_) por onde passam os nossos dados. Podemos ver as classes e objectos do scikit-learn como uma espécie de Legos que podemos montar de formas diversas. Nesta analogia, a classe `Pipeline` é o tabuleiro de montagem, e os vários métodos por onde os nossos dados passam são as peças de legos.

No nosso caso, apenas iremos ter duas etapas: primeiro uma normalização dos dados, usando um objecto da classe `StandardScaler`, seguido de um objecto do tipo `LogisticRegression`.

Para além disso, temos que repartir os nossos dados entre consunto de treino e de teste, e garantir que apenas as colunas com os descritores são usadas pelo modelo.

In [ ]:
from sklearn import pipeline
from sklearn import preprocessing
from sklearn import linear_model
from sklearn import model_selection

# primeiro criamos as peças do modelo
normalizador = preprocessing.StandardScaler()
clf = linear_model.LogisticRegression()

# Agora montamos as peças numa pipeline
modelo = pipeline.Pipeline(steps=[('Norm',normalizador),
                                  ('Clas',clf)])

# definir o nome das variáveis target e features
target = 'Mutagenic'
features = ['MolLogP','BalabanJ','NumHAcceptors','NumHDonors','TPSA','FractionCSP3']

# Partir os dados em treino e teste
treino, teste = model_selection.train_test_split(dados,train_size=0.6,stratify=dados[target],random_state=42)
X_train = treino[features]
y_train = treino[target]
X_test = teste[features]
y_test = teste[target]

# Agora temos tudo para treinar o modelo, usando o método fit()
start=time.time()
modelo.fit(X_train,y_train)
print(f"Treino terminou em {time.time()-start:0.3f} segundos")

O primeiro passo na avaliação do modelo é calcular o _score_ do mesmo nos conjuntos de treino e de teste:

In [ ]:
print(f"Score no conjunto de treino: {modelo.score(X_train,y_train):0.4f}")
print(f"Score no conjunto de teste:  {modelo.score(X_test,y_test):0.4f}")

Os scores não são muito altos, mas ao menos não há uma queda grande do score do teste relativamente ao score do treino. Podemos ver se estes scores são gerados por falsos negativos ou falsos positivos (ou ambos!) com a matriz de confusão.

In [ ]:
# matrix de confusão aplicada ao conjunto total dos dados
from sklearn import metrics
metrics.ConfusionMatrixDisplay.from_estimator(modelo,dados[features],dados[target],normalize='true')

Como podemos verificar, o modelo consegue prever rezoavelmente bem os casos positivos, mas tem dificuldades em prever os casos negativos (elevada fracção de falsos positivos).

### Treino e Avaliação de um Modelo Avançado (_Random Forest_)

Os modelos de _Machine Learning_ são modelos suficientemente complexos para poderem fazer estimações em fronteiras particularmente complexas. O preço a pagar por essa flexibilidade é a tendência para se sobre-ajustarem (_overfitting_) aos dados de treino, ao ponto de perderem capacidade preditiva quando confrontados com novos dados. 

As _Random Forest_ (RF) são um conjunto de métodos tradicionalmente considerados como muito resistentes ao _overfitting_. Para este fim, as RF são compostas por um número definido de árvores de decisão. Cada uma destas árvores define um conjunto de regras a aplicar aos dados de entreada para distinguir entre casos positivos e negativos. No entanto, cada árvore é treinada de forma diferente, quer por alteração dos pesos de cada entrada nos dados de treino, quer por estar limitada apenas a uma fracção das observações ou variáveis do conjunto de treino.

Quando treinamos um modelo RF, este optimiza as regras de cada árvore de forma a ter a melhor capacidade possível de prever as situações no conjunto de treino. Porém, o número de árvores e a fracção de linhas e colunas visíveis por cada árvore têm que ser escolhidas previamente (a estas variáveis damos o nome de **hiper-parametros**). Isto é normalmente feito através de um formalismo de validação cruzada: para cada combinação destes parâmetros, o conjunto de treino é dividido em _n_ partes, e o modelo é treinado _n_ vezes. Em cada uma dessas vezes, uma das partes funciona como conjunto de teste e as outras como conjunto de treino. No final do procedimento temos, para cada conjunto de hiper-parametros, um valor médio do _score_ para o sub-conjunto de teste que nos permite escolher qual a combinação de hiper-parâmetros que confere a melhor capacidade preditiva ao modelo.

No caso do scikit-learn, isto é feito incorporando o nosso modelo num objecto do tipo `GridSearchCV` o qual (graças à opção `refit=True`) faz automaticamente o treino do modelo com os melhores hiper-parametros e usando a totalidade dos dados de treino. O código abaixo funciona de forma análoga ao modelo da regressão logística, tendo-se envolvido o modelo num objecto `GridSearchCV` ao qual também passamos um diccionário com o nome dos hiper-parametros a optimizar e os valores que desejamos ver estudados.

In [ ]:
from sklearn import pipeline
from sklearn import preprocessing
from sklearn import ensemble
from sklearn import model_selection

# primeiro criamos as peças do modelo
normalizador = preprocessing.StandardScaler()
clf = ensemble.RandomForestClassifier(n_estimators=16) # uma floresta com 16 árvores

# Agora montamos as peças numa pipeline
modelo_base = pipeline.Pipeline(steps=[('Norm',normalizador),
                                       ('Clas',clf)])

# definir o nome das variáveis target e features
target = 'Mutagenic'
features = ['MolLogP','BalabanJ','NumHAcceptors','NumHDonors','TPSA','FractionCSP3']

# Definimos quais e quais os valores dos hiper-parametros a estudar
hp = dict()
hp['Clas__max_features']=np.linspace(0.5,1.0,10) # variar nº de features vistas por cada árvore de 50% a 100% em 10 passos
hp['Clas__max_samples']=np.linspace(0.5,1.0,10) # idem, para o nº de pontos de dados

# o nosso modelo final é o modelo_base "embebido" num objecto GridSearchCV

modelo = model_selection.GridSearchCV(modelo_base,
                                      hp,
                                      return_train_score=True,
                                      refit=True)

# Partir os dados em treino e teste
treino, teste = model_selection.train_test_split(dados,train_size=0.6,stratify=dados[target],random_state=42)
X_train = treino[features]
y_train = treino[target]
X_test = teste[features]
y_test = teste[target]

# Agora temos tudo para treinar o modelo, usando o método fit()
start=time.time()
modelo.fit(X_train,y_train)
print(f"Treino terminou em {time.time()-start:0.3f} segundos")

Tal como no caso da Regressão Logístoca, podemos começar por ver os scores e a matriz de confusão.

In [ ]:
print(f"Score no conjunto de treino: {modelo.score(X_train,y_train):0.4f}")
print(f"Score no conjunto de teste:  {modelo.score(X_test,y_test):0.4f}")
from sklearn import metrics
metrics.ConfusionMatrixDisplay.from_estimator(modelo,dados[features],dados[target],normalize='true')

A performance do modelo é significativamente melhor que o modelo linear, mas a quebra no score do conjunto de teste deixa adivinhar algum _overfitting_. Podemos fazer a matriz de confusão só para este conjunto de forma a ter mais informação sobre a sua capacidade preditiva.

In [ ]:
metrics.ConfusionMatrixDisplay.from_estimator(modelo,teste[features],teste[target],normalize='true')

Contrariamente ao modelo da regressão logística, este modelo parece ser mais propenso a fornecer falsos positivos. A opção por um modelo ou pelo outro irá depender, entre outras coisas, do seu contexto de utilização.

Para além da análise da resposta do modelo, podemos também observar como os hiper-parâmetros testados afectam a resposta do modelo.

In [ ]:
cv_data = pd.DataFrame(modelo.cv_results_)[['param_Clas__max_features','param_Clas__max_samples','mean_test_score','mean_train_score']]
cv_data['param_Clas__max_features'] = np.array([float(f"{x:0.2f}") for x in cv_data['param_Clas__max_features'].to_numpy()])
cv_data['param_Clas__max_samples'] = np.array([float(f"{x:0.2f}") for x in cv_data['param_Clas__max_samples'].to_numpy()])
fig, axs = plt.subplots(nrows=2,ncols=2,figsize=(19,13))
sns.boxplot(x='param_Clas__max_features', y='mean_train_score', data=cv_data, ax=axs[0,0], manage_ticks=False)
sns.boxplot(x='param_Clas__max_features', y='mean_test_score', data=cv_data, ax=axs[0,1])
sns.boxplot(x='param_Clas__max_samples', y='mean_train_score', data=cv_data, ax=axs[1,0])
sns.boxplot(x='param_Clas__max_samples', y='mean_test_score', data=cv_data, ax=axs[1,1])

Podemos também obter o valor dos hiper-parametros optimizados:

In [ ]:
print(modelo.best_params_)

Uma análise importante prende-se com a importância de cada _feature_. Isto é feito através do algoritmo de importância por permuta - _Permutation Importance_ - o qual mede a sensibilidade da resposta do modelo a uma corrupção aleatória dos dados de entrada.

In [ ]:
from sklearn import inspection
pi_data = inspection.permutation_importance(modelo,dados[features],dados[target])
pi_data = pd.DataFrame({'Feature':features, 'Mean Importance':pi_data['importances_mean'], 'StD Importance':pi_data['importances_std']})
pi_data.sort_values(by='Mean Importance', ascending=False, inplace=True)
print(pi_data)

Sabemos que BalabanJ, FractionCSP3 e MolLogP são as três variáveis mais importantes. Importa agora saber como a resposta do modelo varia com estas variáveis. Para isto, fazemos gráficos de dependência parcial, os quais mostram a resposta média do modelo em função de uma (ou duas) variáveis, quando todas as outras variáveis são substituídas por ruído.

In [ ]:
fig, axs = plt.subplots(nrows=2,ncols=3,tight_layout=True,figsize=(15,10))
inspection.PartialDependenceDisplay.from_estimator(modelo,dados[features], features=['BalabanJ'], ax=axs[0,0])
inspection.PartialDependenceDisplay.from_estimator(modelo,dados[features], features=['FractionCSP3'], ax=axs[0,1])
inspection.PartialDependenceDisplay.from_estimator(modelo,dados[features], features=['MolLogP'], ax=axs[0,2])
inspection.PartialDependenceDisplay.from_estimator(modelo,dados[features], features=[('BalabanJ','FractionCSP3')], ax=axs[1,0])
inspection.PartialDependenceDisplay.from_estimator(modelo,dados[features], features=[('BalabanJ','MolLogP')], ax=axs[1,1])
inspection.PartialDependenceDisplay.from_estimator(modelo,dados[features], features=[('FractionCSP3','MolLogP')], ax=axs[1,2])

Nos gráficos 2D verificamos que existem multiplas regiões em que a resposta parcial do modelo tende para não mutagénico (a roxo) e regiões onde esta resposta tende para mutagénico (amarelo). Sabendo o significado de cada um destes descritores, podemos racionalizar quais as características que podem conferir carater mutagénico a uma dada molécula.

## Parte 2 - Modelo regressor da actividade antiproliferativa de células HeLa de derivados de xanteno.

A segunda parte do TP5 usa dados originalmente publicados em:

Zukić, S.; Maran, U. "Modelling of antiproliferative activity measured in HeLa cervical cancer cells in a series of xanthene derivatives." _SAR QSAR Environ. Res._ 2020, **31**, 905-921. http://dx.doi.org/10.1080/1062936X.2020.1839131

Neste caso, a actividade antiproliferativa dos derivados de xanteno é uma variável contínua, pIC50, onde IC50 é a metade da concentraão inibitória máxima. Quanto maior a capacidade inibitória, menor o IC50, e maior o pIC50. De notar também que esta base de dados usa InChI em vez de SMILES. 

Tal como na Parte I, esta parte do trabalho começa pelo treino e avaliação de um modelo linear, seguindo-se a o treino e avaliação de um modelo de Machine Learning. A célula seguinte gera os dados iniciais e carrega os pacotes necessários.

In [ ]:
import time
import numpy as np
import pandas as pd
import seaborn as sns

import matplotlib.pyplot as plt

from rdkit import Chem
from rdkit.Chem import Descriptors
from rdkit.Chem import Draw

with open('parte2-dados.csv','w') as f:
  f.write("""Compound Id;pIC50;InChI
1;5.28;InChI=1S/C19H11BrO6/c20-8-1-2-12(21)9(3-8)19-10-4-13(22)15(24)6-17(10)26-18-7-16(25)14(23)5-11(18)19/h1-7,21-24H
2;5.05;InChI=1S/C20H14O7/c1-26-16-4-2-3-9(20(16)25)19-10-5-12(21)14(23)7-17(10)27-18-8-15(24)13(22)6-11(18)19/h2-8,21-23,25H,1H3
3;4.56;InChI=1S/C19H12O7/c20-11-2-1-8(3-12(11)21)19-9-4-13(22)15(24)6-17(9)26-18-7-16(25)14(23)5-10(18)19/h1-7,20-24H
4;5.31;InChI=1S/C21H16O7/c1-26-17-4-3-10(5-20(17)27-2)21-11-6-13(22)15(24)8-18(11)28-19-9-16(25)14(23)7-12(19)21/h3-9,22-24H,1-2H3
6;4.44;InChI=1S/C20H13NO9/c1-29-18-3-8(2-11(20(18)26)21(27)28)19-9-4-12(22)14(24)6-16(9)30-17-7-15(25)13(23)5-10(17)19/h2-7,22-24,26H,1H3
7;5.54;InChI=1S/C21H16O6/c1-2-26-12-5-3-11(4-6-12)21-13-7-15(22)17(24)9-19(13)27-20-10-18(25)16(23)8-14(20)21/h3-10,22-24H,2H2,1H3
8;5.3;InChI=1S/C21H17NO5/c1-22(2)12-5-3-11(4-6-12)21-13-7-15(23)17(25)9-19(13)27-20-10-18(26)16(24)8-14(20)21/h3-10,23-25H,1-2H3
9;6.15;InChI=1S/C20H11F3O5/c21-20(22,23)10-3-1-9(2-4-10)19-11-5-13(24)15(26)7-17(11)28-18-8-16(27)14(25)6-12(18)19/h1-8,24-26H
10;4.39;InChI=1S/C21H15NO6/c1-10(23)22-12-4-2-11(3-5-12)21-13-6-15(24)17(26)8-19(13)28-20-9-18(27)16(25)7-14(20)21/h2-9,24-26H,1H3,(H,22,23)
11;5.82;InChI=1S/C19H11BrO5/c20-10-3-1-2-9(4-10)19-11-5-13(21)15(23)7-17(11)25-18-8-16(24)14(22)6-12(18)19/h1-8,21-23H
13;4.76;InChI=1S/C19H11ClO5/c20-12-4-2-1-3-9(12)19-10-5-13(21)15(23)7-17(10)25-18-8-16(24)14(22)6-11(18)19/h1-8,21-23H
14;5.55;InChI=1S/C19H11ClO5/c20-10-3-1-9(2-4-10)19-11-5-13(21)15(23)7-17(11)25-18-8-16(24)14(22)6-12(18)19/h1-8,21-23H
15;5.17;InChI=1S/C19H11FO5/c20-10-3-1-9(2-4-10)19-11-5-13(21)15(23)7-17(11)25-18-8-16(24)14(22)6-12(18)19/h1-8,21-23H
17;5.14;InChI=1S/C19H11ClO5/c20-10-3-1-2-9(4-10)19-11-5-13(21)15(23)7-17(11)25-18-8-16(24)14(22)6-12(18)19/h1-8,21-23H
18;4.04;InChI=1S/C19H11NO8/c21-12-2-1-8(20(26)27)3-9(12)19-10-4-13(22)15(24)6-17(10)28-18-7-16(25)14(23)5-11(18)19/h1-7,21-24H
20;4.38;InChI=1S/C19H11NO7/c21-13-5-11-17(7-15(13)23)27-18-8-16(24)14(22)6-12(18)19(11)9-1-3-10(4-2-9)20(25)26/h1-8,21-23H
24;4.01;InChI=1S/C24H28O5/c1-23(2)9-14(25)20-17(11-23)29-18-12-24(3,4)10-15(26)21(18)19(20)13-7-6-8-16(28-5)22(13)27/h6-8,19,27H,9-12H2,1-5H3
34;5.25;InChI=1S/C24H27BrO3/c1-13-6-7-14(15(25)8-13)20-21-16(26)9-23(2,3)11-18(21)28-19-12-24(4,5)10-17(27)22(19)20/h6-8,20H,9-12H2,1-5H3
36;4.2;InChI=1S/C23H24Br2O3/c1-22(2)8-15(26)20-17(10-22)28-18-11-23(3,4)9-16(27)21(18)19(20)12-5-13(24)7-14(25)6-12/h5-7,19H,8-11H2,1-4H3
38;4.21;InChI=1S/C23H25BrO4/c1-22(2)8-15(26)20-17(10-22)28-18-11-23(3,4)9-16(27)21(18)19(20)13-7-12(24)5-6-14(13)25/h5-7,19,25H,8-11H2,1-4H3
42;4.6;InChI=1S/C26H32O6/c1-25(2)10-15(27)21-19(12-25)32-20-13-26(3,4)11-16(28)22(20)24(21)23-17(30-6)8-14(29-5)9-18(23)31-7/h8-9,24H,10-13H2,1-7H3
44;4.57;InChI=1S/C25H30O5/c1-24(2)10-16(26)22-19(12-24)30-20-13-25(3,4)11-17(27)23(20)21(22)15-9-14(28-5)7-8-18(15)29-6/h7-9,21H,10-13H2,1-6H3
12;5.34;InChI=1S/C19H10ClFO5/c20-10-2-1-3-11(21)19(10)18-8-4-12(22)14(24)6-16(8)26-17-7-15(25)13(23)5-9(17)18/h1-7,22-24H
16;4.44;InChI=1S/C20H14O6/c1-25-17-5-3-2-4-10(17)20-11-6-13(21)15(23)8-18(11)26-19-9-16(24)14(22)7-12(19)20/h2-9,21-23H,1H3
19;5.23;InChI=1S/C19H12O5/c20-13-6-11-17(8-15(13)22)24-18-9-16(23)14(21)7-12(18)19(11)10-4-2-1-3-5-10/h1-9,20-22H
23;4.06;InChI=1S/C23H26O5/c1-22(2)8-15(26)20-17(10-22)28-18-11-23(3,4)9-16(27)21(18)19(20)12-5-6-13(24)14(25)7-12/h5-7,19,24-25H,8-11H2,1-4H3
40;4.61;InChI=1S/C25H29BrO5/c1-24(2)9-15(27)21-18(11-24)31-19-12-25(3,4)10-16(28)22(19)20(21)14-7-13(26)8-17(29-5)23(14)30-6/h7-8,20H,9-12H2,1-6H3
""")

### Geração dos Descritores
A geração dos descritores é feita de modo semelhrante ao protobolo estabelecido na Parte 1, mas com uma lista de descritores substancialmente maior.

In [ ]:
dados = pd.read_csv('parte2-dados.csv',sep=';')
# criar colunas vazias para os descritores
dados['MolLogP'] = np.zeros(len(dados))
dados['MolMR'] = np.zeros(len(dados))
dados['BalabanJ'] = np.zeros(len(dados))
dados['BertzCT'] = np.zeros(len(dados))
dados['NumHAcceptors'] = np.zeros(len(dados))
dados['NumHDonors'] = np.zeros(len(dados))
dados['TPSA'] = np.zeros(len(dados))
dados['FractionCSP3'] = np.zeros(len(dados))
dados['NOCount'] = np.zeros(len(dados))
dados['NumValenceElectrons'] = np.zeros(len(dados))
dados['NumAromaticRings'] = np.zeros(len(dados))
dados['LabuteASA'] = np.zeros(len(dados))
dados['NumRotatableBonds'] = np.zeros(len(dados))
dados['MinPartialCharge'] = np.zeros(len(dados))
dados['MaxPartialCharge'] = np.zeros(len(dados))
dados['MinEStateIndex'] = np.zeros(len(dados))
dados['MaxEStateIndex'] = np.zeros(len(dados))

for i,line in dados.iterrows():
  m = Chem.MolFromInchi(line['InChI'])
  dados.loc[i,'MolLogP'] = Descriptors.MolLogP(m)
  dados.loc[i,'MolMR'] = Descriptors.MolMR(m)
  dados.loc[i,'BalabanJ'] = Descriptors.BalabanJ(m)
  dados.loc[i,'NumHAcceptors'] = Descriptors.NumHAcceptors(m)
  dados.loc[i,'NumHDonors'] = Descriptors.NumHDonors(m)
  dados.loc[i,'TPSA'] = Descriptors.TPSA(m)
  dados.loc[i,'FractionCSP3'] = Descriptors.FractionCSP3(m)
  dados.loc[i,'NOCount'] = Descriptors.NOCount(m)
  dados.loc[i,'NumValenceElectrons'] = Descriptors.NumValenceElectrons(m)
  dados.loc[i,'NumAromaticRings'] = Descriptors.NumAromaticRings(m)
  dados.loc[i,'LabuteASA'] = Descriptors.LabuteASA(m)
  dados.loc[i,'BertzCT'] = Descriptors.BertzCT(m)
  dados.loc[i,'NumRotatableBonds'] = Descriptors.NumRotatableBonds(m)
  dados.loc[i,'MinPartialCharge'] = Descriptors.MinPartialCharge(m)
  dados.loc[i,'MaxPartialCharge'] = Descriptors.MaxPartialCharge(m)
  dados.loc[i,'MinEStateIndex'] = Descriptors.MinEStateIndex(m)
  dados.loc[i,'MaxEStateIndex'] = Descriptors.MaxEStateIndex(m)

dados

Da análise dos descritores, fazemos apenas a matriz de correlação:

In [ ]:
# Heatmap da matriz de correlação
descritores=['MolLogP','MolMR','LabuteASA','NumRotatableBonds',
             'MinEStateIndex','MaxEStateIndex','MinPartialCharge',
             'MaxPartialCharge','BertzCT','BalabanJ','NumHAcceptors',
             'NumHDonors','TPSA','FractionCSP3','NOCount','NumValenceElectrons',
             'NumAromaticRings']
_, ax = plt.subplots(figsize=(12,12))
sns.heatmap(dados[descritores+['pIC50']].corr(), cmap="RdYlBu", annot=True, ax=ax)

### Treino e Avaliação de um Modelo Linear (Regressão Multilinear/Método dos Mínimos Quadrados)

O método dos mínimos quadrados está disponível no scikit-learn, no sub-módulo `linear_regression`. O procedimento para este modelo é análogo ao da regressão logística na parte 1:

In [ ]:
from sklearn import pipeline
from sklearn import preprocessing
from sklearn import linear_model
from sklearn import model_selection

# primeiro criamos as peças do modelo
normalizador = preprocessing.StandardScaler()
reg = linear_model.LinearRegression()

# Agora montamos as peças numa pipeline
modelo = pipeline.Pipeline(steps=[#('Norm',normalizador),
                                  ('Clas',reg)])

# definir o nome das variáveis target e features
target = 'pIC50'
features = descritores

# Partir os dados em treino e teste
treino, teste = model_selection.train_test_split(dados,train_size=0.7,random_state=42)
X_train = treino[features]
y_train = treino[target]
X_test = teste[features]
y_test = teste[target]

# Agora temos tudo para treinar o modelo, usando o método fit()
start=time.time()
modelo.fit(X_train,y_train)
print(f"Treino terminou em {time.time()-start:0.3f} segundos")

Podemos agora fazer a avaliação dos scores ($r^2$) do modelo.

In [ ]:
print(f"Score no conjunto de treino: {modelo.score(X_train,y_train):0.4f}")
print(f"Score no conjunto de teste:  {modelo.score(X_test,y_test):0.4f}")

Note-se que o $r^2$ usado para a avaliação do ajuste de regressão não é o $r^2$ de Pearson, que varia entre 0 e 1. Esta variante de $r^2$ varia entre $-∞$ e 1, sendo que 1 significa um ajuste perfeito, e a aussência de um limite inferior deriva da ideia que um modelo pode ser sempre infinitamente mau a fazer previsões.

No nosso caso, estamos perante um caso severo de overfitting, em que o modelo se ajusta perfeitamente aos dados de treino, e dá previsões péssimas para o conjunto de teste. Podemos ver isso graficamente através do gráfico de ajuste (_fitness plot_).

In [ ]:
y_train_pred = modelo.predict(X_train)
y_test_pred = modelo.predict(X_test)
fig, ax = plt.subplots()
l_train = ax.scatter(y_train,y_train_pred)
l_test = ax.scatter(y_test,y_test_pred)
y_full=np.concatenate((y_test,y_train))
l3 = ax.plot(y_full,y_full)
ax.set_xlabel(f"{target} (Original)")
ax.set_ylabel(f"{target} (Predicted)")
l = ax.legend([l_train,l_test],['Train data','Test data'])

### Treino e Avaliação de um Modelo Avançado (_Kernel Ridge Regression_)
O método _Kernel Ridge Regression_ (KRR) é um método que ajusta os dados fornecidos através de uma manipulação interna das coordenadas. O tipo de manipulação é o _kernel_ e corresponde a uma trnasformação dos pontos no espaço. O ajuste linear é depois feito nesse espaço transformado. Neste exemplo, iremos usar o _kernel_ `rbf` o qual toma aforma approximada de uma gaussiana em torno de cada par de pontos. Para este modelo, devemos ajustar dois hiper-parametros: `alpha` (o coeficiente de regularização) e `gamma` (a largura de cada função do kernel).

In [ ]:
from sklearn import pipeline
from sklearn import preprocessing
from sklearn import kernel_ridge

# primeiro criamos as peças do modelo
normalizador = preprocessing.StandardScaler()

reg = kernel_ridge.KernelRidge(kernel='rbf')

# Agora montamos as peças numa pipeline
modelo_base = pipeline.Pipeline(steps=[#('Norm',normalizador),
                                       ('Reg',reg)])

# definir o nome das variáveis target e features
target = 'pIC50'
features = descritores

# Definimos quais e quais os valores dos hiper-parametros a estudar
hp = dict()
hp['Reg__alpha']=np.logspace(-10,-5,20) 
hp['Reg__gamma']=np.logspace(-10,-5,20)
#hp['Reg__degree']=[2,3,4] 
# o nosso modelo final é o modelo_base "embebido" num objecto GridSearchCV

modelo = model_selection.GridSearchCV(modelo_base,
                                      hp,
                                      return_train_score=True,
                                      refit=True)

# Partir os dados em treino e teste
treino, teste = model_selection.train_test_split(dados,train_size=0.7,random_state=42)
X_train = treino[features]
y_train = treino[target]
X_test = teste[features]
y_test = teste[target]

# Agora temos tudo para treinar o modelo, usando o método fit()
start=time.time()
modelo.fit(X_train,y_train)
print(f"Treino terminou em {time.time()-start:0.3f} segundos")

Fazendo agora a avaliação do modelo:

In [ ]:
print(f"Score no conjunto de treino: {modelo.score(X_train,y_train):0.4f}")
print(f"Score no conjunto de teste:  {modelo.score(X_test,y_test):0.4f}")

In [ ]:
y_train_pred = modelo.predict(X_train)
y_test_pred = modelo.predict(X_test)
fig, ax = plt.subplots()
l_train = ax.scatter(y_train,y_train_pred)
l_test = ax.scatter(y_test,y_test_pred)
y_full=np.concatenate((y_test,y_train))
l3 = ax.plot(y_full,y_full)
ax.set_xlabel(f"{target} (Original)")
ax.set_ylabel(f"{target} (Predicted)")
l = ax.legend([l_train,l_test],['Train data','Test data'])

Apesar da melhoria da performance em relação ao modelo linear simples, o score no conjunto de treino continua a ser demasiado baixo para podermos considerar este modelo como preditivo. 

## Questões
1. Dos quatro modelos treinados neste trabalho, qual o que tem melhor performance?
1. Com base nos resultados dos quatro modelos treinados, comente a seguinte afirmação "O uso de técnicas avançadas de Machine Learning só garante bons resultados na presença de grandes quantidades de dados".
1. Considere apenas os dois modelos classificadores (Parte I). Qual o tipo de erro mais prevalente em cada um deles?
1. Considerando a sua resposta à pergunta anterior, indique qual destes modelos (se algum) é preferível para determinar que compostos deverão ser estudados mais aprofundadamente em estudos _in vivo_? Justifique.
1. No seguimento da pergunta anterior, qual dos modelos classificadores (se algum) poderá ser usado para determinar a introdução de uma dada substância no mercado, com base na sua capacidade mutagénica? Justifique.
1. Comente a seguinte afirmação: "O modelo regressor KRR é totalmente inútil".
